In [1]:
import pandas as pd
import pytz

In [2]:
full_df = pd.read_csv("store status.csv")

In [3]:
df = full_df.iloc[:7]

In [4]:
ndf = full_df.copy()
ndf['timestamp_utc'] = pd.to_datetime(ndf['timestamp_utc'])

In [15]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse
# Assuming you have a DataFrame named 'df' with columns 'store_id', 'status', and 'timestamp_utc'

# PostgreSQL connection details
db_user = 'postgres'
db_password = 'password'
db_host = "localhost"
db_port = 5432
db_name = 'Restaurant_info'
encoded_password = urllib.parse.quote_plus(db_password)
# Create the PostgreSQL connection string

connection_string = f'postgresql+psycopg2://{db_user}:{encoded_password}@{db_host}:{db_port}/{db_name}'

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Table name in the database
table_name = 'status_info'

# Upload the DataFrame to the PostgreSQL database
ndf.to_sql(table_name, engine, if_exists='replace', index=False)

# Close the database connection
engine.dispose()


In [36]:
current_time = pd.to_datetime("2023-01-24 10:59:59.47922 UTC")
start_of_last_hour = current_time - pd.DateOffset(hours=1)
start_of_last_hour = pd.to_datetime(start_of_last_hour)
mask = (ndf['store_id'] == 4430370444829587327) & (ndf['timestamp_utc'] >= start_of_last_hour)
filtered_df = ndf[mask]
filtered_df

,store_id,status,timestamp_utc
117227,4430370444829587327,active,2023-01-24 16:10:17.438134+00:00
156207,4430370444829587327,active,2023-01-24 11:02:53.128058+00:00
178541,4430370444829587327,active,2023-01-25 02:02:01.363972+00:00
203314,4430370444829587327,active,2023-01-25 16:05:54.650172+00:00
217233,4430370444829587327,active,2023-01-25 14:13:19.736460+00:00
218880,4430370444829587327,active,2023-01-25 01:05:52.525522+00:00
261078,4430370444829587327,active,2023-01-24 10:10:32.280714+00:00
265991,4430370444829587327,active,2023-01-25 10:08:37.096873+00:00
344294,4430370444829587327,active,2023-01-24 22:01:46.100007+00:00
396268,4430370444829587327,active,2023-01-25 00:07:08.880003+00:00


In [ ]:
# Traverse through each store id then gather it's uptimes and downtimes corresponding to 
# store ids and then use extrapolation logic to calculate uptime and downtime. 
# Also take note of the corresponding timezones with utc. 

In [95]:
timezones = pd.read_csv('bq-results-20230125-202210-1674678181880.csv')
active = pd.read_csv("Menu hours.csv")


In [61]:
#timezones
timezonefr = timezones[timezones['store_id'].eq(8139926242460185114)]['timezone_str'].values[0]
timezonefr


'Asia/Beirut'

In [83]:
gt =  timezones[timezones['store_id'].eq(1774428314218854399)]['timezone_str'].values[0]
gt

'America/New_York'

In [96]:

for index, row in active.iterrows():
    store_id = row['store_id']
    timezone_fr = timezones[timezones['store_id'] == store_id]
    if timezone_fr.empty:
        timezone = timezone_fr['America/Denver']
    else:
        timezone = timezone_fr['timezone_str'].values[0]
    store_timezone = pytz.timezone(timezone)
    stlocal = pd.to_datetime(row['start_time_local'])
    edlocal = pd.to_datetime(row['end_time_local'])
    
    localized_end_time = store_timezone.localize(edlocal)    
    localized_start_time = store_timezone.localize(stlocal)

    localized_end_time = localized_end_time.astimezone(pytz.UTC)
    localized_start_time = localized_start_time.astimezone(pytz.UTC)
    active.at[index, 'start_time_local'] = localized_start_time.strftime('%H:%M:%S')
    active.at[index,'end_time_local']=localized_end_time.strftime('%H:%M:%S')


In [97]:
active

,store_id,day,start_time_local,end_time_local
0,1481966498820158979,4,2023-06-04 04:00:00,2023-06-04 04:10:00
1,1481966498820158979,2,2023-06-04 04:00:00,2023-06-04 04:10:00
2,1481966498820158979,0,2023-06-04 04:00:00,2023-06-04 04:10:00
3,1481966498820158979,1,2023-06-04 04:00:00,2023-06-04 04:10:00
4,1481966498820158979,5,2023-06-04 04:00:00,2023-06-04 04:10:00
...,...,...,...,...
86193,1774428314218854399,4,2023-06-04 04:00:00,2023-06-05 03:59:59
86194,1774428314218854399,6,2023-06-04 04:00:00,2023-06-05 03:59:59
86195,1774428314218854399,3,2023-06-04 04:00:00,2023-06-05 03:59:59
86196,1774428314218854399,2,2023-06-04 04:00:00,2023-06-05 03:59:59
